<a href="https://colab.research.google.com/github/rafaellabastos/savvyfix-ia-notebook/blob/rm552425/precificacao_dinamica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SavvyFix - Precificação dinâmica**

<p>2TDSPF</p>

<p>Douglas Magalhães de Araujo - 552008</p>
<p>Gustavo Argüello Bertacci - 551304</p>
<p>Luiz Fillipe Farias - 99519</p>
<p>Rafaella Monique do Carmo Bastos - 552425</p>

# **Dataset**

<p>Fonte: https://www.kaggle.com/code/jeffersonvalandro/analisando-inflacao-brasil/input?select=relatorio_preco_medio_mensal_2023.csv</p>

## Descrição do conjunto de dados
<p>O objetivo desse notebook é criar um modelo de Machine Learning utilizando a técnica de Regressão Linear para demonstrar os prováveis preços dinamizados ao longo dos meses com base nos meses anteriores.</p>
<p>Os dados foram coletados a partir da inflação dos produtos ao longo dos meses do ano de 2023.</p>
<p>Observação: com o tempo, este modelo será alimentado com dados a partir da localização, clima, demanda, procura e horário.</p>

## **Importação das bibliotecas**

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

## **Inicialização do Data Frame**

In [73]:
dados = pd.read_csv('relatorio_preco_medio_mensal_2023.csv')
dados.head()

,Produto/Unidade,Nível de Comercialização,U.F.,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023
0,ABACATE (kg),ATACADO,RR,"6,83","6,67","6,83","7,00","7,00","6,00","6,00","6,60","7,00","7,00","7,00","7,00"
1,ABACAXI HAVAÍ (kg),PRODUTOR,PR,"1,85","2,31","2,30","2,30","2,30",NaN,NaN,NaN,NaN,NaN,"2,60","2,60"
2,ABACAXI (kg),ATACADO,RR,"2,59","2,65","2,79","6,00","8,00","4,50","4,50","4,95","5,00","7,50","7,00","7,50"
3,ABACAXI PÉROLA (6.50 kg),PRODUTOR,DF,"21,80","20,14","23,45","26,65","34,82","36,87","25,47","27,36","28,48","31,92","25,05","25,75"
4,ABACAXI PÉROLA (kg),NaN,AM,"3,81","3,75","3,75","3,90","4,00","4,25","4,02","3,94","4,27","4,25","4,51","4,23"


## **Visualização e tratamento do DataSet**

---



In [74]:
# Visualização das últimas linhas
dados.tail()

,Produto/Unidade,Nível de Comercialização,U.F.,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023
2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2026,Fonte: Companhia Nacional de Abastecimento - C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027,"As informações podem ser reproduzidas, desde q...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028,Proibida a comercialização das informações dis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# Quantidade de linhas e colunas do DataFrame
dados.shape

(2029, 15)

In [76]:
# Nomes das colunas
dados.columns

Index(['Produto/Unidade', 'Nível de Comercialização', 'U.F.', '01/2023',
       '02/2023', '03/2023', '04/2023', '05/2023', '06/2023', '07/2023',
       '08/2023', '09/2023', '10/2023', '11/2023', '12/2023'],
      dtype='object')

In [77]:
# Informação do formato de dados das colunas
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Produto/Unidade           551 non-null    object
 1   Nível de Comercialização  309 non-null    object
 2   U.F.                      2024 non-null   object
 3   01/2023                   1901 non-null   object
 4   02/2023                   1902 non-null   object
 5   03/2023                   1874 non-null   object
 6   04/2023                   1874 non-null   object
 7   05/2023                   1884 non-null   object
 8   06/2023                   1879 non-null   object
 9   07/2023                   1852 non-null   object
 10  08/2023                   1781 non-null   object
 11  09/2023                   1765 non-null   object
 12  10/2023                   1706 non-null   object
 13  11/2023                   1603 non-null   object
 14  12/2023                 

In [78]:
# Exclusão de colunas sem importância
dados = dados.drop('Nível de Comercialização', axis=1)
dados = dados.drop('U.F.', axis=1)
dados.head()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023
0,ABACATE (kg),"6,83","6,67","6,83","7,00","7,00","6,00","6,00","6,60","7,00","7,00","7,00","7,00"
1,ABACAXI HAVAÍ (kg),"1,85","2,31","2,30","2,30","2,30",NaN,NaN,NaN,NaN,NaN,"2,60","2,60"
2,ABACAXI (kg),"2,59","2,65","2,79","6,00","8,00","4,50","4,50","4,95","5,00","7,50","7,00","7,50"
3,ABACAXI PÉROLA (6.50 kg),"21,80","20,14","23,45","26,65","34,82","36,87","25,47","27,36","28,48","31,92","25,05","25,75"
4,ABACAXI PÉROLA (kg),"3,81","3,75","3,75","3,90","4,00","4,25","4,02","3,94","4,27","4,25","4,51","4,23"


In [79]:
# Duplicação da coluna Produto/Unidade
dados['Produto/Unidade_2'] = dados['Produto/Unidade']
dados.head()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,Produto/Unidade_2
0,ABACATE (kg),"6,83","6,67","6,83","7,00","7,00","6,00","6,00","6,60","7,00","7,00","7,00","7,00",ABACATE (kg)
1,ABACAXI HAVAÍ (kg),"1,85","2,31","2,30","2,30","2,30",NaN,NaN,NaN,NaN,NaN,"2,60","2,60",ABACAXI HAVAÍ (kg)
2,ABACAXI (kg),"2,59","2,65","2,79","6,00","8,00","4,50","4,50","4,95","5,00","7,50","7,00","7,50",ABACAXI (kg)
3,ABACAXI PÉROLA (6.50 kg),"21,80","20,14","23,45","26,65","34,82","36,87","25,47","27,36","28,48","31,92","25,05","25,75",ABACAXI PÉROLA (6.50 kg)
4,ABACAXI PÉROLA (kg),"3,81","3,75","3,75","3,90","4,00","4,25","4,02","3,94","4,27","4,25","4,51","4,23",ABACAXI PÉROLA (kg)


In [62]:
# Conversão de uma coluna de dados categóricos para dados numéricos
le = LabelEncoder()

for column in dados.columns:
  if column!= 'Produto/Unidade' and dados[column].dtype == 'object':
    dados[column] = le.fit_transform(dados[column])

dados.head()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,Produto/Unidade_2
0,ABACATE (kg),1120,1137,1118,1135,1214,1099,1068,1090,1177,1140,1109,1055,0
1,ABACAXI HAVAÍ (kg),43,505,479,447,473,1461,1420,1405,1386,1317,468,436,2
2,ABACAXI (kg),520,527,511,1039,1312,881,848,881,938,1167,1109,1083,1
3,ABACAXI PÉROLA (6.50 kg),570,557,591,608,805,825,635,657,654,710,559,526,3
4,ABACAXI PÉROLA (kg),771,775,752,715,849,862,815,735,818,770,768,723,4


In [11]:
# Informação do formato de dados das colunas pós-transformação
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Produto/Unidade    551 non-null    object
 1   01/2023            2029 non-null   int64 
 2   02/2023            2029 non-null   int64 
 3   03/2023            2029 non-null   int64 
 4   04/2023            2029 non-null   int64 
 5   05/2023            2029 non-null   int64 
 6   06/2023            2029 non-null   int64 
 7   07/2023            2029 non-null   int64 
 8   08/2023            2029 non-null   int64 
 9   09/2023            2029 non-null   int64 
 10  10/2023            2029 non-null   int64 
 11  11/2023            2029 non-null   int64 
 12  12/2023            2029 non-null   int64 
 13  Produto/Unidade_2  2029 non-null   int64 
dtypes: int64(13), object(1)
memory usage: 222.0+ KB


In [12]:
# Exclusão de linhas com valores Nan
dados = dados.dropna()
dados = dados.iloc[:-3]

In [13]:
# Visualização do DataFrame sem as linhas com NaN
dados.tail()

,Produto/Unidade,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,Produto/Unidade_2
2011,UVA INDÚSTRIA NIÁGARA (kg),894,883,1475,1426,997,999,994,990,1002,1041,1046,979,538
2012,UVA MOSCATO (kg),486,495,463,436,40,44,41,40,42,37,44,38,540
2013,UVA NIÁGARA (kg),28,29,20,24,19,20,21,18,17,16,20,11,541
2014,VACA GORDA (@),636,606,541,507,445,381,404,1405,1386,1317,1276,1249,543
2015,VACA GORDA (15 kg),692,692,619,532,578,435,410,1405,1386,1317,1276,1249,542


## **Análises preliminares**

In [14]:
# Estátistica descritiva
dados.describe()

,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,Produto/Unidade_2
count,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000
mean,779.029197,791.337591,806.906934,796.622263,832.313869,818.748175,792.890511,812.609489,793.098540,779.722628,778.808394,760.381387,273.421533
std,438.847934,446.146434,448.135276,438.123314,454.234044,457.949686,445.848191,455.072597,460.282003,443.896656,424.694429,414.431333,158.586868
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,442.750000,450.000000,463.750000,454.000000,484.000000,446.250000,438.750000,440.750000,400.750000,395.000000,414.250000,411.000000,136.750000
50%,768.500000,793.000000,811.000000,810.500000,849.000000,817.500000,777.000000,811.000000,789.000000,788.500000,799.500000,789.000000,272.500000
75%,1104.500000,1137.500000,1174.750000,1164.250000,1228.250000,1243.000000,1206.250000,1273.250000,1273.000000,1263.250000,1252.250000,1233.000000,410.250000
max,1479.000000,1500.000000,1475.000000,1426.000000,1469.000000,1461.000000,1420.000000,1405.000000,1386.000000,1317.000000,1276.000000,1249.000000,548.000000


In [15]:
# Matriz de correlação
remocao = dados.drop('Produto/Unidade', axis=1)
matriz = remocao.corr().round(4)
print(matriz)

                   01/2023  02/2023  03/2023  04/2023  05/2023  06/2023  \
01/2023             1.0000   0.7734   0.6871   0.6597   0.5568   0.5841   
02/2023             0.7734   1.0000   0.8163   0.7078   0.6328   0.6306   
03/2023             0.6871   0.8163   1.0000   0.8319   0.7202   0.6780   
04/2023             0.6597   0.7078   0.8319   1.0000   0.8684   0.7666   
05/2023             0.5568   0.6328   0.7202   0.8684   1.0000   0.8142   
06/2023             0.5841   0.6306   0.6780   0.7666   0.8142   1.0000   
07/2023             0.5731   0.5760   0.6459   0.7476   0.7797   0.8663   
08/2023             0.5211   0.5258   0.5908   0.6473   0.6792   0.7457   
09/2023             0.4452   0.4391   0.5015   0.5403   0.5640   0.6179   
10/2023             0.4048   0.4027   0.5038   0.4833   0.4791   0.5237   
11/2023             0.3364   0.3320   0.4119   0.3866   0.3931   0.3784   
12/2023             0.3575   0.3402   0.4131   0.3922   0.3643   0.3376   
Produto/Unidade_2   0.022

## **Análises gráficas**